In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds

In [2]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets["train"], datasets["test"]

**Defining the MirroredStrategy**

In [3]:
strategy = tf.distribute.MirroredStrategy()
NUM_GPUS = strategy.num_replicas_in_sync #stores the number of GPUs available on this VM.
print(NUM_GPUS)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


1


In [4]:
num_train_examples = info.splits["train"].num_examples
num_train_examples = info.splits["test"].num_examples

**Your batch size must be equal to the batch_size_per_worker * total number of workers.**

In [5]:
BUFFER_SIZE = 1000

BATCH_SIZE_PER_REPLICA = 128
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_GPUS

In [6]:
def scale(image, label):
  image = tf.cast(image, tf.float16)
  image /= 255. 

  return image, label

In [7]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

**Declaring the model within the strategy scope!** 

In [8]:
with strategy.scope():
  clf = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
                                    tf.keras.layers.MaxPooling2D(), 
                                    tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(64, activation="relu"), 
                                    tf.keras.layers.Dense(10, activation="softmax")
  ])

In [9]:
clf.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


**You are supposed to see the strategy related log messages at the beginning of the training if you successfully enabled the distributed training strategy.**

In [10]:
clf.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


469/469 [==============================] - 12s 18ms/step - loss: 0.5105 - accuracy: 0.8601 - val_loss: 0.1084 - val_accuracy: 0.9664
Epoch 2/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0949 - accuracy: 0.9733 - val_loss: 0.0665 - val_accuracy: 0.9778
Epoch 3/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0634 - accuracy: 0.9820 - val_loss: 0.0541 - val_accuracy: 0.9811
Epoch 4/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0495 - accuracy: 0.9853 - val_loss: 0.0474 - val_accuracy: 0.9829
Epoch 5/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0390 - accuracy: 0.9888 - val_loss: 0.0446 - val_accuracy: 0.9844
Epoch 6/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0323 - accuracy: 0.9905 - val_loss: 0.0476 - val_accuracy: 0.9840
Epoch 7/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0261 - accuracy: 0.9927 - val_loss: 0.0475 - val_accuracy: 0.9837
Epoch 8/20
469/